In [13]:
import pstats

import torch

In [14]:
a = torch.arange(12).reshape(2, 6)
a

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])

In [15]:
torch.chunk(a, 2, dim=-1)

(tensor([[0, 1, 2],
         [6, 7, 8]]),
 tensor([[ 3,  4,  5],
         [ 9, 10, 11]]))

In [16]:
p = pstats.Stats("profile.txt")
p.sort_stats("cumulative").print_stats(10)

Thu Jan  8 18:45:54 2026    profile.txt

         147852761 function calls (146200170 primitive calls) in 151.930 seconds

   Ordered by: cumulative time
   List reduced from 9169 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12021    0.108    0.000  104.509    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:727(__next__)
    12021    0.303    0.000  103.572    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:786(_next_data)
    12001    0.378    0.000  102.921    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\_utils\fetch.py:47(fetch)
  1200100    9.760    0.000  100.506    0.000 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torchvision\datasets\mnist.py:131(__getitem__)
  1200100    0.719    0.000   67.317    0.000 C:\Users\otto\repos\02476-mlops-project\.venv\Lib

In [17]:
p = pstats.Stats("profile_2.txt")
p.sort_stats("cumulative").print_stats(10)

Thu Jan  8 18:52:25 2026    profile_2.txt

         19458704 function calls (19006276 primitive calls) in 52.182 seconds

   Ordered by: cumulative time
   List reduced from 9149 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12000    0.066    0.000   13.687    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\_tensor.py:570(backward)
    12000    0.106    0.000   13.618    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\autograd\__init__.py:243(backward)
    12000    0.056    0.000   12.912    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\autograd\graph.py:832(_engine_run_backward)
    12000   12.846    0.001   12.846    0.001 {method 'run_backward' of 'torch._C._EngineBase' objects}
    12021    0.061    0.000    8.357    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:727(__next__)
    120

In [18]:
import torch
import torchvision.models as models
from torch.profiler import ProfilerActivity, profile

model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    model(inputs)

In [19]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.15%     104.600us        70.22%      48.150ms       2.408ms            20  
                aten::convolution         0.34%     232.300us        70.07%      48.046ms       2.402ms            20  
               aten::_convolution         0.21%     147.000us        69.73%      47.813ms       2.391ms            20  
         aten::mkldnn_convolution        68.93%      47.268ms        69.52%      47.666ms       2.383ms            20  
                 aten::max_pool2d         0.02%      14.200us        14.35%       9.842ms       9.842ms             1  
    aten::max_pool2d_with_indices       

In [20]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=30))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                     aten::conv2d         0.04%      25.500us        17.40%      11.930ms       2.983ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], []]  
                aten::convolution         0.07%      51.100us        17.36%      11.905ms       2.976ms             4                     [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], [], 

In [21]:
a = torch.randn(10,20)
a.shape

torch.Size([10, 20])

In [27]:
a.argmax(dim=1)

tensor([12, 10, 15,  3, 19,  1, 14, 13,  4,  2])